# 月历图可视化

## 请选择年份

In [1]:
year = 2010

## 请选择表格

In [2]:
print("可以选择的表格有：")
print([table for table in cfps[year]])

可以选择的表格有：
['adult', 'child', 'comm', 'famconf', 'famecon']


In [3]:
table = 'adult'

## 请选择变量

In [30]:
import re
# 筛选包含日期的变量
variables = {k:v["key"] for k,v in cfps[year][table].schema.items() if re.match(r"^(?=.*日期).+（[年月日]）$", v["key"])}
real_variables = set(v[:-3] for v in variables.values())
infos = {k:{} for k in real_variables }
for real_v in real_variables: # 找出日期对应的年和月
    for k, v in variables.items():
        if v.startswith(real_v):
            if re.match(".+（月）$", v):
                infos[real_v]['month'] = k
            elif re.match(".+（年）$", v):
                infos[real_v]['year'] = k
var_list = list(filter(lambda x: len(x[1])==2, infos.items())) # 筛选出年和月都有的变量
print('可以选择的变量有：')
print(
    '\n'.join(
        f"{i+1}. {k} `{v['year']}` 年 `{v['month']}` 月" for i, (k, v) in enumerate(var_list)
    )
)
print("请选择变量（输入选项前面的序号）:")

可以选择的变量有：
1. 姓名10：出生日期 `qb303y_a_10` 年 `qb303m_a_10` 月
2. 您刚过世的配偶去世的日期是 `qe507y` 年 `qe507m` 月
3. 姓名15：出生日期 `qb303y_a_15` 年 `qb303m_a_15` 月
4. 姓名11：出生日期 `qb303y_a_11` 年 `qb303m_a_11` 月
5. 姓名13：出生日期 `qb303y_a_13` 年 `qb303m_a_13` 月
6. 姓名9：出生日期 `qb303y_a_9` 年 `qb303m_a_9` 月
7. 姓名7：出生日期 `qb303y_a_7` 年 `qb303m_a_7` 月
8. 姓名4：出生日期 `qb303y_a_4` 年 `qb303m_a_4` 月
9. 姓名2：出生日期 `qb303y_a_2` 年 `qb303m_a_2` 月
10. 您与刚过世的配偶的结婚日期是 `qe501y` 年 `qe501m` 月
11. 姓名8：出生日期 `qb303y_a_8` 年 `qb303m_a_8` 月
12. 请问您的出生日期 `qa1y` 年 `qa1m` 月
13. 姓名12：出生日期 `qb303y_a_12` 年 `qb303m_a_12` 月
14. 姓名14：出生日期 `qb303y_a_14` 年 `qb303m_a_14` 月
15. 您与初婚配偶的结婚日期是 `qe605y` 年 `qe605m` 月
16. 请问您与前任配偶的结婚日期是 `qe405y` 年 `qe405m` 月
17. 姓名6：出生日期 `qb303y_a_6` 年 `qb303m_a_6` 月
18. 您与现在的配偶的结婚日期是 `qe210y` 年 `qe210m` 月
19. 姓名5：出生日期 `qb303y_a_5` 年 `qb303m_a_5` 月
20. 姓名3：出生日期 `qb303y_a_3` 年 `qb303m_a_3` 月
21. 姓名1：出生日期 `qb303y_a_1` 年 `qb303m_a_1` 月
请选择变量（输入选项前面的序号）:


In [23]:
no = 21 # 选择变量（输入选项前面的序号）

## 可视化

In [25]:
months = ['一月', '二月' , '三月', '四月', '五月', '六月', '七月', '八月', '九月', '十月', '十一月', '十二月']
friendly_name, var = var_list[no-1]
data = sql(f"select {var['year']}, {var['month']} from {table}_{year} where {var['month']}>0 and {var['year']}>0;")
years = set(data[var['year']]) # 在数据中出现过的年份
counts = data.groupby([var['year'], var['month']]).size().reset_index(name="count") # 计数
min_count=min(counts['count'])
max_count=max(counts['count'])

In [29]:
def convert_to_native_number(column):
    return column.item() if len(column)==1 else 0
(
    HeatMap()
    .add_xaxis([str(year) for year in years])
    .add_yaxis(
        f"`{friendly_name}`的每月数目",
        months,
        [
            [
                str(year), 
                i-1, 
                convert_to_native_number(
                    counts[
                        (counts[var['year']]==year) & (counts[var['month']]==i)
                    ]['count']
                )
            ] 
            for year in years for i in range(1,12+1)
        ],
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title=f"`{friendly_name}` 月历图统计", 
            pos_left='center'
        ),
        visualmap_opts=opts.VisualMapOpts(
            orient='horizontal', 
            pos_left='center', 
            min_=0, 
            max_=max_count
        ),
        legend_opts=opts.LegendOpts(
            pos_top="6%"
        )
    )
    .render_notebook()
)